[Covid-19-Dashboard](https://github.com/salta-ak/Covid-19-Dashboard) (C) Saltanat Akhmet, 2020 ([s.akhmet@se20.qmul.ac.uk])(mailto:s.akhmet@se20.qmul.ac.uk). All rights reserved.

# Covid-19 Dashboard

In [122]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [123]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [126]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
jsondata={}
filters = [ 'areaType=overview']
structure = {"date": "date", "cases": "newCasesByPublishDate","hospital": "newAdmissions", "deaths": "cumDeaths28DaysByDeathDateRate" }
api = Cov19API(filters=filters, structure=structure)
timeseries=api.get_json()
jsondata=timeseries['data']    
jsondata_ga={}
filters_ga = [ 'areaType=nation', 'areaName=England']
structure_ga = {"males": "maleCases","females": "femaleCases"}
api_age = Cov19API(filters=filters_ga, structure=structure_ga)
agedist=api_age.get_json()
jsondata_ga=agedist['data']
with open("timeseries.json", "wt") as OUTF:
    json.dump(jsondata, OUTF)
with open("agedistribution.json", "wt") as OUTF:
    json.dump(jsondata_ga, OUTF)

In [132]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def wrangle_data(rawdata,rawdata_ga):
    dates=[dictionary['date'] for dictionary in rawdata ]
    dates.sort()
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    index=pd.date_range(startdate, enddate, freq='D')
    timeseriesdf=pd.DataFrame(index=index, columns=['cases', 'hospital', 'deaths'])
    for entry in rawdata:
        date=parse_date(entry['date'])
        for column in ['cases', 'hospital', 'deaths']:
            if pd.isna(timeseriesdf.loc[date, column]):
                value= float(entry[column]) if entry[column]!=None else 0.0
                timeseriesdf.loc[date, column]=value
    timeseriesdf.fillna(0.0, inplace=True)
    

    datadic=rawdata_ga[0] # data['data'] is a list
    males=datadic['males']
    females=datadic['females']
    ageranges=[x['age'] for x in males]
    def min_age(agerange):
        agerange=agerange.replace('+','') # remove the + from 90+
        start=agerange.split('_')[0]
        return int(start)
    ageranges.sort(key=min_age)
    age_df=pd.DataFrame(index=ageranges, columns=['males','females', 'total'])
    for entry in males:
        ageband=entry['age'] # our index position
        age_df.loc[ageband, 'males']=entry['rate']
    for entry in females:
        ageband=entry['age']
        age_df.loc[ageband, 'females']=entry['rate']
    
# this is straightforward
    age_df['total']=age_df['males']+age_df['females']
    return timeseriesdf , age_df
# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
with open("timeseries.json", "rt") as INFILE:
    jsondata=json.load(INFILE)
with open("agedistribution.json", "rt") as INFILE:
    jsondata_ga=json.load(INFILE)
df,df_ga =wrangle_data(jsondata,jsondata_ga) # df is the dataframe for plotting
df.to_pickle("timeseriesdf.pkl")
df_ga.to_pickle("agedf.pkl")

In [128]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    filters = [ 'areaType=overview']
    structure = {"date": "date", "cases": "newCasesByPublishDate","hospital": "newAdmissions", "deaths": "cumDeaths28DaysByDeathDateRate" }
    api = Cov19API(filters=filters, structure=structure)
    timeseries=api.get_json()
    jsondata=timeseries['data']  
    
    filters_ga = [ 'areaType=nation', 'areaName=England']
    structure_ga = {"males": "maleCases","females": "femaleCases"}
    api_age = Cov19API(filters=filters_ga, structure=structure_ga)
    agedist=api_age.get_json()
    jsondata_ga=agedist['data']
    return jsondata, jsondata_ga  
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
     # return data read from the API

In [129]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata,apidata_ga=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global df
    df,df_ga=wrangle_data(apidata,apidata_ga)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
    apibutton.icon="check"
    # apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Update data', # you may want to change this...
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='exclamation-triangle'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

Button(button_style='success', description='Update data', icon='exclamation-triangle', style=ButtonStyle(), to…

## Graphs and Analysis

The line graph shows the number of new covid cases, new hospital admissions and 28 days cummulutive death rates in England. 
The bar chart shows the rate of cases by age and gender in England. 

In [130]:
series=wdg.SelectMultiple(
    options=['cases', 'hospital', 'deaths'],
    value=['cases', 'hospital', 'deaths'],
    rows=3,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False
)

# try replacing HBox with a VBox
controls=wdg.HBox([series, scale])
def timeseries_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        df[list(gcols)].plot(logy=logscale)
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=series.value
    if current==series.options[0]:
        other=series.options[0:3]
    else:
        other=series.options[0:3]
    series.value=other # forces the redraw
    series.value=current # now we can change it back
    
graph=wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scale})
    
display(controls, graph)

Output()

In [131]:
agecols=wdg.SelectMultiple(
    options=['males', 'females', 'total'], # options available
    value=['males', 'females'], # initial value
    rows=3, # rows of the selection box
    description='Sex',
    disabled=False
)

def age_graph(graphcolumns):
    # our callback function.
    ncols=len(graphcolumns)
    if ncols>0:
        df_ga.plot(kind='bar', y=list(graphcolumns)) # graphcolumns is a tuple - we need a list
    else:
        # if the user has not selected any column, print a message instead
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
 

output=wdg.interactive_output(age_graph, {'graphcolumns': agecols})

display(agecols, output)

SelectMultiple(description='Sex', index=(0, 1), options=('males', 'females', 'total'), rows=3, value=('males',…

Output()

data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*